## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_path_list = glob.glob('/kaggle/input/119-googleq-a-train-3fold-bert-base-unc-category/' + '*.h5')
model_path_list.sort()
print('Models to predict:', model_path_list)

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict: ['/kaggle/input/119-googleq-a-train-3fold-bert-base-unc-category/model_fold_1.h5', '/kaggle/input/119-googleq-a-train-3fold-bert-base-unc-category/model_fold_2.h5', '/kaggle/input/119-googleq-a-train-3fold-bert-base-unc-category/model_fold_3.h5']
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map misspellings
    test[feature] = test[feature].apply(lambda x: map_misspellings(x))
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512
N_CLASS_CAT = test['category'].nunique()

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH, trainable=True)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output_seq = Dense(N_CLASS, activation="sigmoid", name="output_seq")(x)
    output_class = Dense(N_CLASS_CAT, activation="softmax", name="output_class")(pooled_output)
    output = Concatenate(name="output")([output_seq, output_class])

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test)[: ,:N_CLASS] / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.932925,0.653193,0.185822,0.474512,0.573977,0.644520,0.700929,0.637675,0.643116,...,0.891661,0.906010,0.539005,0.959990,0.960548,0.840683,0.108871,0.047286,0.825363,0.909538
1,46,0.870863,0.486075,0.004624,0.732363,0.769898,0.944881,0.566029,0.442789,0.093637,...,0.711332,0.962617,0.658982,0.972679,0.983295,0.882039,0.943737,0.096838,0.091012,0.878535
2,70,0.908037,0.663179,0.042633,0.812662,0.785886,0.911039,0.615316,0.452686,0.113531,...,0.845233,0.950023,0.614679,0.975628,0.975944,0.904158,0.171986,0.053176,0.873134,0.926706
3,132,0.834363,0.385480,0.008088,0.593922,0.724006,0.933258,0.565725,0.389816,0.139930,...,0.613742,0.943084,0.593490,0.959014,0.981929,0.881263,0.887472,0.169254,0.371699,0.877646
4,200,0.912649,0.437745,0.038065,0.803424,0.729722,0.843051,0.680569,0.558479,0.154017,...,0.669134,0.934880,0.667192,0.963699,0.977260,0.852178,0.251891,0.111493,0.702812,0.916234


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.866249,0.565831,0.033932,0.676878,0.770331,0.856454,0.598651,0.456868,0.246683,...,0.760644,0.939271,0.655250,0.960676,0.976127,0.879079,0.575768,0.127094,0.497887,0.901027
std,2812.670060,0.054383,0.131893,0.058649,0.111002,0.110835,0.124061,0.050958,0.085923,0.195128,...,0.096091,0.019981,0.043986,0.011031,0.008295,0.033737,0.325514,0.059587,0.282955,0.021787
min,39.000000,0.693447,0.295242,0.003325,0.215208,0.288550,0.248535,0.485205,0.324295,0.009394,...,0.570249,0.824419,0.539005,0.912651,0.932021,0.731830,0.011580,0.009415,0.021205,0.836604
25%,2572.000000,0.825389,0.451943,0.007818,0.609057,0.720892,0.832238,0.560202,0.395897,0.092685,...,0.673209,0.928273,0.625697,0.953768,0.971625,0.858785,0.241141,0.083325,0.246971,0.885831
50%,5093.000000,0.867571,0.560549,0.011944,0.671727,0.775458,0.905004,0.594833,0.435551,0.181358,...,0.767485,0.943041,0.655160,0.962079,0.977746,0.883504,0.685975,0.131597,0.484866,0.902653
75%,7482.000000,0.909734,0.666483,0.026122,0.751163,0.847108,0.931661,0.633402,0.492616,0.363863,...,0.846205,0.953231,0.682151,0.968154,0.982229,0.901551,0.862194,0.168640,0.742328,0.917818
max,9640.000000,0.971716,0.877349,0.542563,0.963191,0.970438,0.975663,0.733693,0.729356,0.828884,...,0.953040,0.982434,0.813656,0.984387,0.990959,0.966001,0.976917,0.343787,0.992450,0.948531
